In [4]:
!nvidia-smi

Sun Mar 17 23:29:51 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:0A:00.0  On |                  N/A |
|  0%   48C    P5             24W /  215W |     893MiB /   8192MiB |      9%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import opendatasets as od
import string
import random
import os

from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Normalize, ToTensor

In [3]:
if torch.cuda.is_available():
    device=torch.device(type="cuda", index=0)
else:
    device=torch.device(type="cpu", index=0)
print(device)

cuda:0


In [ ]:
class ShakespeareDataset(Dataset):
    def __init__(self, file_path) -> None:
        super().__init__()
        self.data = self.loadData(file_path)
        self.vocab = self.getVocab()
        
    def loadData(self, file_path):
        # Preprocess as well
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read()
        lines = data.split('\n')
        lines = [line.strip(r'\"') for line in lines]
        words = ' '.join(lines).split()
        punct = set(string.punctuation)
        words = [''.join([char for char in list(word) if char not in punct]) for word in words]
        return words
    
    def getVocab(self):
        char_counts = Counter(self.data)
        vocab = {char: idx for idx, (char, _) in enumerate(sorted(char_counts.items()))}
        vocab_size = len(vocab)
        return vocab, vocab_size
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        

In [4]:
with open ('Datasets/shakespear_dataset/alllines.txt', 'r') as f:
    doc = f.read()

In [8]:
# get list of words from document
def doc2words(doc):
    lines = doc.split('\n')
    lines = [line.strip(r'\"') for line in lines]
    words = ' '.join(lines).split()
    return words



def removepunct(words):
    punct = set(string.punctuation)
    words = [''.join([char for char in list(word) if char not in punct]) for word in words]
    return words

def getvocab(words):
    wordfreq = Counter(words)
    sorted_wordfreq = sorted(wordfreq, key=wordfreq.get)
    return sorted_wordfreq

# get dictionary of int to words and word to int
def vocab_map(vocab):
    int_to_vocab = {k:w for k,w in enumerate(vocab)}
    vocab_to_int = {w:k for k,w in int_to_vocab.items()}
    return int_to_vocab, vocab_to_int

In [9]:
words = removepunct(doc2words(doc))
vocab = getvocab(words)
# print(vocab)
int_to_vocab, vocab_to_int = vocab_map(vocab)
print(vocab_to_int)

{'shortwinded': 0, 'strands': 1, 'trenching': 2, 'illsheathed': 3, 'impressed': 4, 'naild': 5, 'forwarding': 6, 'Leading': 7, 'Welshwomen': 8, 'Brake': 9, 'Holyrood': 10, 'Archibald': 11, 'evervaliant': 12, 'Balkd': 13, 'Holmedons': 14, 'Athol': 15, 'Murray': 16, 'Angus': 17, 'Menteith': 18, 'straightest': 19, 'nighttripping': 20, 'cradleclothes': 21, 'Malevolent': 22, 'WALES': 23, 'fatwitted': 24, 'unbuttoning': 25, 'leapinghouses': 26, 'hethat': 27, 'gracemajesty': 28, 'snatched': 29, 'ridge': 30, 'apparentBut': 31, 'lugged': 32, 'Lincolnshire': 33, 'Moorditch': 34, 'rascalliest': 35, 'pursetaking': 36, 'Sugar': 37, 'soldest': 38, 'Madeira': 39, 'Yedward': 40, 'chops': 41, 'Allhallown': 42, 'waylaid': 43, 'immask': 44, 'incomprehensible': 45, 'unyoked': 46, 'mists': 47, 'falsify': 48, 'attract': 49, 'Redeeming': 50, 'Danger': 51, 'North': 52, 'misprison': 53, 'trimly': 54, 'stubbleland': 55, 'pouncetbox': 56, 'slovenly': 57, 'smarting': 58, 'popinjay': 59, 'neglectingly': 60, 'wounds